In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt 
import scipy
import os
import glob
import seaborn as sns
import itertools
import gc
import pickle
from nptdms import TdmsFile
from scipy import stats
from natsort import natsorted
from tqdm import tqdm

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import FormatStrFormatter
import matplotlib as mpl
import scienceplots

matplotlib.rcParams['text.usetex'] = True
plt.rcParams["text.usetex"] = True
# import seaborn as sns
from scipy.stats import norm
from scipy.interpolate import interp1d
from scipy.interpolate import make_interp_spline, BSpline

np.random.seed(16)
matplotlib.rcParams['figure.dpi'] = 300

plt.style.use(['science','nature'])
plt.rcParams['figure.constrained_layout.use'] = True

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(["Open","Arc","Short","Normal"])

import sys
sys.path.append('../.')
# sys.path.extend(
#     [
#         os.path.join(os.path.dirname(os.path.abspath(__file__)),".."),
#     ]
# )
# from pydub import AudioSegment
import itertools
import string
import glob
import subprocess
import seedir
from utils.preprocessing import one_column,two_column,cm_std,cm_bright,cm_mark,cm_highCon,centimeter
from utils.preprocessing import print_tdms_structure,split_pulses_by_interval,getShiftMean
from utils.MLUtils import TesterBase
mpl.rcParams.update(two_column)

from MyDataset import get_dataset, get_collate_fn
from MyModels import GrindingPredictor

alphabet = list(string.ascii_lowercase)
sampling_rate = 10000000

CHECKPOINT_DIR = os.path.join("../lfs","checkpoints")
HIS_DIR = os.path.join("../lfs","train_his")
cpus = 2

allowed_input_types=[
    'ae_spec',
    'ae_features',
    'ae_features+pp',
    'ae_spec+ae_features',

    'vib_spec',
    'vib_features',
    'vib_features+pp',
    'vib_spec+vib_features',

    'ae_spec+ae_features+vib_spec+vib_features',

    'all',
]

In [15]:
folds=10
repeat=10
for _input_type in allowed_input_types:
    for i in range(int(folds*repeat)):
        file_path = f"{_input_type}_fold{i}_of_folds{folds}.pt"
        file_path = os.path.join(HIS_DIR, file_path)

In [16]:
file_path

'../lfs/train_his/all_fold99_of_folds10.pt'

In [17]:
dataset = get_dataset(input_type=_input_type,  cpus=cpus)
collate_fn = get_collate_fn(input_type=_input_type)
gc.collect()

model = GrindingPredictor(interp=False,input_type=_input_type)

Required components: {'all'}


In [19]:
dataset[0]

/mnt/c/Users/Public/Documents/LocalWork/Smart_Grinding/postprocessing/.././MyDataset.py:163: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  item['features_ae'] = torch.tensor([


{'label': tensor(112),
 'features_pp': tensor([0.7284, 0.0000, 0.0000]),
 'features_ae': tensor([[0.0413, 0.0378, 0.0134,  ..., 0.0065, 0.0226, 0.0190],
         [0.0390, 0.0307, 0.0167,  ..., 0.0084, 0.0486, 0.0395],
         [0.5556, 0.4444, 0.6667,  ..., 0.3333, 0.4444, 0.4444],
         [0.4615, 0.3846, 0.4615,  ..., 0.2308, 0.4615, 0.4615]]),
 'spec_ae': tensor([[[[-25.9288, -33.4342, -36.8104,  ..., -48.0844, -34.8646, -46.1319],
           [-28.1438, -40.2344, -27.2027,  ..., -19.7401, -20.0636, -31.5557],
           [-23.7362, -27.8281, -19.2636,  ..., -16.8388, -20.7722, -26.6928],
           ...,
           [-78.9217, -80.0000, -78.0530,  ..., -79.2395, -80.0000, -79.6818],
           [-80.0000, -80.0000, -80.0000,  ..., -80.0000, -80.0000, -77.8523],
           [-80.0000, -80.0000, -80.0000,  ..., -80.0000, -79.1219, -77.9345]],
 
          [[-16.4466, -42.7402, -22.1990,  ..., -46.2477, -28.9176, -24.1672],
           [-10.2349, -18.3327, -21.4609,  ..., -25.4187, -20.3929,

In [ ]:
class RoughnessTester(TesterBase):
    """
    A specialized tester for analyzing surface roughness (Ra) predictions.
    
    Inherits from TesterBase and adds a custom method to plot time-series
    predictions with a color-coded background indicating the machining regime
    (e.g., ductile vs. brittle).
    """
    def __init__(self, model, device, forward_fn):
        # Initialize the base class for a regression task
        super().__init__(model=model, device=device, task_type='regression', forward_fn=forward_fn)

    def plot_timeseries_with_regimes(self, feature_name='BDI', ductile_threshold=1.0):
        """
        Creates a time-series plot of predicted vs. true Ra with a color-coded background
        representing the machining regime.

        Args:
            feature_name (str): The name of the feature in the input data that
                                determines the regime (e.g., 'BDI').
            ductile_threshold (float): The threshold for the feature to distinguish
                                       between ductile and brittle modes.
        """
        if self.predictions is None or self.labels is None or self.inputs is None:
            raise RuntimeError("You must run `predict(return_inputs=True)` before calling this method.")
        
        # Extract the regime feature from the stored inputs
        try:
            regime_feature = np.concatenate([batch[feature_name] for batch in self.inputs], axis=0).flatten()
        except KeyError:
            raise KeyError(f"Feature '{feature_name}' not found in the provided dataset inputs. "
                           "Ensure it's part of the data returned by the dataloader.")

        mae = mean_absolute_error(self.labels, self.predictions)
        time_steps = np.arange(len(self.labels))

        plt.style.use('seaborn-v0_8-whitegrid')
        fig, ax = plt.subplots(figsize=(15, 6))

        # Plot predicted and true values
        ax.plot(time_steps, self.labels, label='Measured Ground Truth ($R_a$)', color='black', linewidth=2)
        ax.plot(time_steps, self.predictions, label=f'Model Prediction ($R_a$)', color='darkorange', linestyle='--', linewidth=2.5)

        # Add the color-coded background for regimes
        ax.fill_between(time_steps, ax.get_ylim()[0], ax.get_ylim()[1],
                        where=regime_feature > ductile_threshold,
                        color='blue', alpha=0.2, label='Ductile Dominated (BDI > 1)')
        ax.fill_between(time_steps, ax.get_ylim()[0], ax.get_ylim()[1],
                        where=regime_feature <= ductile_threshold,
                        color='red', alpha=0.2, label='Brittle Dominated (BDI <= 1)')

        ax.set_xlabel('Time Step or Sample Number', fontsize=12)
        ax.set_ylabel('Surface Roughness ($R_a$)', fontsize=12)
        ax.set_title('Model Prediction Fidelity Across Machining Regimes', fontsize=14, fontweight='bold')
        ax.legend(loc='upper right')
        ax.set_xlim(0, len(time_steps))

        # Add caption insight as text on the plot
        caption = f"The model demonstrates high fidelity in predicting $R_a$ (MAE = {mae:.3f}).\n" \
                  "Prediction accuracy remains robust during transitions between ductile (blue) and brittle (red) regimes."
        fig.text(0.5, -0.02, caption, ha='center', fontsize=10, style='italic')
        
        plt.tight_layout()
        plt.show()
